In [60]:
# standard imports
import numpy as np
import pandas as pd
from capston_db_conn import db_conn

In [61]:
conn = db_conn()
print(conn)

<connection object at 0x7fd7e88ce580; dsn: 'user=postgres password=xxx dbname=capstone host=mads-capstone-db.codt5pemnxi8.us-east-1.rds.amazonaws.com', closed: 0>


In [62]:
rf_sql = """
SELECT *
FROM redfin_county_full;
"""

redfin = pd.read_sql(rf_sql, con=conn, parse_dates=["period_begin", "period_end"])
# conn.close()


/Users/ianbyrne/opt/anaconda3/envs/capstone/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [63]:
redfin.head()

,county_fips,period_begin,period_end,period_duration,region_type,region_type_id,table_id,is_seasonally_adjusted,region,city,...,price_drops,price_drops_mom,price_drops_yoy,off_market_in_two_weeks,off_market_in_two_weeks_mom,off_market_in_two_weeks_yoy,parent_metro_region,parent_metro_region_metro_code,last_updated,county
0,08014,2012-04-01,2012-04-30,30,county,5,368,f,"Broomfield County, CO",None,...,0.153846,-0.20979,0.060096,1.000000,1.000000,1.000000,"Denver, CO",19740.0,2022-01-09 14:29:56,Broomfield County
1,13011,2018-05-01,2018-05-31,30,county,5,509,f,"Banks County, GA",None,...,NaN,NaN,NaN,0.111111,0.111111,-0.031746,Georgia nonmetropolitan area,NaN,2022-01-09 14:29:56,Banks County
2,39107,2019-01-01,2019-01-31,30,county,5,2213,f,"Mercer County, OH",None,...,NaN,NaN,NaN,0.000000,0.000000,0.000000,"Celina, OH",16380.0,2022-01-09 14:29:56,Mercer County
3,39089,2018-11-01,2018-11-30,30,county,5,2204,f,"Licking County, OH",None,...,NaN,NaN,NaN,0.230769,-0.076923,-0.144231,"Columbus, OH",18140.0,2022-01-09 14:29:56,Licking County
4,44005,2013-08-01,2013-08-31,30,county,5,2430,f,"Newport County, RI",None,...,0.025547,NaN,NaN,0.279070,-0.003539,0.035827,"Providence, RI",39300.0,2022-01-09 14:29:56,Newport County


Also grab monthly unemployment by state, state income tax to add to the redfin data.

In [64]:
unem = """
SELECT date,
       rate,
       state,
       fips
FROM fred_state_monthly_ur
WHERE date >= '1/1/12'::date;
"""

unemployment = pd.read_sql(unem, con=conn, parse_dates='date')

/Users/ianbyrne/opt/anaconda3/envs/capstone/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [65]:
unemployment["year"] = unemployment["date"].dt.year
unemployment.rename(columns={"rate": "unemployment_rate"}, inplace=True)
unemployment.head()

,date,unemployment_rate,state,fips,year
0,2012-01-01,7.1,TX,48,2012
1,2012-02-01,7.0,TX,48,2012
2,2012-03-01,6.7,TX,48,2012
3,2012-04-01,6.8,TX,48,2012
4,2012-05-01,6.8,TX,48,2012


In [66]:
unemployment.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6050 entries, 0 to 6049
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   date               6050 non-null   datetime64[ns]
 1   unemployment_rate  6050 non-null   float64       
 2   state              6050 non-null   object        
 3   fips               6050 non-null   int64         
 4   year               6050 non-null   int64         
dtypes: datetime64[ns](1), float64(1), int64(2), object(1)
memory usage: 236.5+ KB


In [67]:
tax_q = """
SELECT sitl.state,
       sitl.state_fips,
       income_tax_low,
       income_tax_high,
       scitl.corp_income_tax_low,
       scitl.corp_income_tax_high,
       sitl.year
FROM state_income_tax_long sitl
         INNER JOIN state_corp_income_tax_long scitl
                    ON sitl.year = scitl.year AND sitl.state_fips = scitl.state_fips
WHERE sitl.year >= 2012;
"""

tax = pd.read_sql(tax_q, con=conn)

/Users/ianbyrne/opt/anaconda3/envs/capstone/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [68]:
tax.head()

,state,state_fips,income_tax_low,income_tax_high,corp_income_tax_low,corp_income_tax_high,year
0,ALABAMA,01,2.00,5.00,6.500,6.500,2012
1,ALASKA,02,0.00,0.00,1.000,9.400,2012
2,ARIZONA,04,2.59,4.54,6.968,6.968,2012
3,ARKANSAS,05,1.00,7.00,1.000,6.500,2012
4,CALIFORNIA,06,1.00,9.30,8.840,8.840,2012


In [69]:
tax["state_fips"] = tax["state_fips"].astype("int64")
tax.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 561 entries, 0 to 560
Data columns (total 7 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   state                 561 non-null    object 
 1   state_fips            561 non-null    int64  
 2   income_tax_low        561 non-null    float64
 3   income_tax_high       561 non-null    float64
 4   corp_income_tax_low   561 non-null    float64
 5   corp_income_tax_high  561 non-null    float64
 6   year                  561 non-null    int64  
dtypes: float64(4), int64(2), object(1)
memory usage: 30.8+ KB


In [70]:
redfin.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 563122 entries, 0 to 563121
Data columns (total 60 columns):
 #   Column                          Non-Null Count   Dtype         
---  ------                          --------------   -----         
 0   county_fips                     563122 non-null  object        
 1   period_begin                    563122 non-null  datetime64[ns]
 2   period_end                      563122 non-null  datetime64[ns]
 3   period_duration                 563122 non-null  int64         
 4   region_type                     563122 non-null  object        
 5   region_type_id                  563122 non-null  int64         
 6   table_id                        563122 non-null  int64         
 7   is_seasonally_adjusted          563122 non-null  object        
 8   region                          563122 non-null  object        
 9   city                            0 non-null       object        
 10  state                           563122 non-null  object 

In [71]:
conn.close()

In [72]:
redfin.period_begin.max()

Timestamp('2021-12-01 00:00:00')

In [73]:
# df1 = redfin.groupby(["county_fips", "state", "property_type", pd.Grouper(key="period_end", freq="M")]).sum()
# df2 = df1.reset_index().groupby(["county_fips", "state", "property_type"]).sum().apply(lambda x: x.asfreq("M")).reset_index()

In [74]:
# create a row for every month and every year for the groupby statements
# redfin.set_index("period_end").groupby(["county_fips", "state", "property_type"]).resample("M").asfreq()

In [75]:
df = redfin.fillna(0)

In [76]:
df.head()

,county_fips,period_begin,period_end,period_duration,region_type,region_type_id,table_id,is_seasonally_adjusted,region,city,...,price_drops,price_drops_mom,price_drops_yoy,off_market_in_two_weeks,off_market_in_two_weeks_mom,off_market_in_two_weeks_yoy,parent_metro_region,parent_metro_region_metro_code,last_updated,county
0,08014,2012-04-01,2012-04-30,30,county,5,368,f,"Broomfield County, CO",0,...,0.153846,-0.20979,0.060096,1.000000,1.000000,1.000000,"Denver, CO",19740.0,2022-01-09 14:29:56,Broomfield County
1,13011,2018-05-01,2018-05-31,30,county,5,509,f,"Banks County, GA",0,...,0.000000,0.00000,0.000000,0.111111,0.111111,-0.031746,Georgia nonmetropolitan area,0.0,2022-01-09 14:29:56,Banks County
2,39107,2019-01-01,2019-01-31,30,county,5,2213,f,"Mercer County, OH",0,...,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,"Celina, OH",16380.0,2022-01-09 14:29:56,Mercer County
3,39089,2018-11-01,2018-11-30,30,county,5,2204,f,"Licking County, OH",0,...,0.000000,0.00000,0.000000,0.230769,-0.076923,-0.144231,"Columbus, OH",18140.0,2022-01-09 14:29:56,Licking County
4,44005,2013-08-01,2013-08-31,30,county,5,2430,f,"Newport County, RI",0,...,0.025547,0.00000,0.000000,0.279070,-0.003539,0.035827,"Providence, RI",39300.0,2022-01-09 14:29:56,Newport County


In [77]:
df2 = unemployment.merge(tax, left_on=["fips", "year"], right_on=["state_fips", "year"])

In [78]:
df2.head()

,date,unemployment_rate,state_x,fips,year,state_y,state_fips,income_tax_low,income_tax_high,corp_income_tax_low,corp_income_tax_high
0,2012-01-01,7.1,TX,48,2012,TEXAS,48,0.0,0.0,0.0,0.0
1,2012-02-01,7.0,TX,48,2012,TEXAS,48,0.0,0.0,0.0,0.0
2,2012-03-01,6.7,TX,48,2012,TEXAS,48,0.0,0.0,0.0,0.0
3,2012-04-01,6.8,TX,48,2012,TEXAS,48,0.0,0.0,0.0,0.0
4,2012-05-01,6.8,TX,48,2012,TEXAS,48,0.0,0.0,0.0,0.0


In [79]:
data = df.merge(df2, left_on=["period_begin", "state_code"], right_on=["date", "state_x"])

In [81]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 562522 entries, 0 to 562521
Data columns (total 71 columns):
 #   Column                          Non-Null Count   Dtype         
---  ------                          --------------   -----         
 0   county_fips                     562522 non-null  object        
 1   period_begin                    562522 non-null  datetime64[ns]
 2   period_end                      562522 non-null  datetime64[ns]
 3   period_duration                 562522 non-null  int64         
 4   region_type                     562522 non-null  object        
 5   region_type_id                  562522 non-null  int64         
 6   table_id                        562522 non-null  int64         
 7   is_seasonally_adjusted          562522 non-null  object        
 8   region                          562522 non-null  object        
 9   city                            562522 non-null  int64         
 10  state                           562522 non-null  object 

In [82]:
data.to_csv("nn_monthly_data.csv.gz", 
           index=False, 
           compression="gzip")

In [83]:
data.info(memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
Int64Index: 562522 entries, 0 to 562521
Data columns (total 71 columns):
 #   Column                          Non-Null Count   Dtype         
---  ------                          --------------   -----         
 0   county_fips                     562522 non-null  object        
 1   period_begin                    562522 non-null  datetime64[ns]
 2   period_end                      562522 non-null  datetime64[ns]
 3   period_duration                 562522 non-null  int64         
 4   region_type                     562522 non-null  object        
 5   region_type_id                  562522 non-null  int64         
 6   table_id                        562522 non-null  int64         
 7   is_seasonally_adjusted          562522 non-null  object        
 8   region                          562522 non-null  object        
 9   city                            562522 non-null  int64         
 10  state                           562522 non-null  object 